In [9]:
!pip install h2o #uncomment to install on first-run
#!pip install --upgrade git+https://github.com/statsmodels/statsmodels
import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use(style = 'ggplot')
plt.rcParams['figure.figsize'] = (10,6)

h2o.init(max_mem_size = "10G")


# This will prompt for authorization.
from google.colab import drive

# Mount your drive. It will be at this path: "/content/gdrive/My Drive/"
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/ml_project/matt
from preproc import custom_pre_proc

#!cat '/gdrive/My Drive/ml_project/preproc.py'
train = pd.read_csv('/content/drive/My Drive/ml_project/matt/train.csv')
test = pd.read_csv('/content/drive/My Drive/ml_project/matt/test.csv')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 mins 41 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.3
H2O cluster version age:,20 days
H2O cluster name:,H2O_from_python_unknownUser_j627fu
H2O cluster total nodes:,1
H2O cluster free memory:,10 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ml_project/matt


In [7]:
#@title
#%cd /content/drive/My\ Drive/ml_project/Katy/Scripts
#from FeatureAdd import nafix, thewholeshabang, beefup, q_to_cat

/content/drive/My Drive/ml_project/Katy/Scripts


FileNotFoundError: ignored

In [0]:
####################
#### MATT DFS ######
####################
train_mattcat = train.copy() ; custom_pre_proc(train_mattcat) 
test_mattcat = test.copy() ; custom_pre_proc(test_mattcat)
data = pd.concat([train_mattcat, test_mattcat], axis = 0)
data = pd.get_dummies(data = data, drop_first=True)
train_mattcat = data.iloc[:1460,:]
test_mattcat = data.iloc[1460:,:]
train_mattcat.to_csv('/content/drive/My Drive/ml_project/train_mattcat.csv')
test_mattcat.to_csv('/content/drive/My Drive/ml_project/test_mattcat.csv')

In [0]:
#@title
####################
#### TRAIN DFS #####
####################
df = train.copy()

df_dumcat = train.copy() ; df_dumcat = nafix(df_dumcat); df_dumcat = pd.get_dummies(df_dumcat, drop_first = True)
df_nocat = train.copy() ; df_nocat = df_nocat._get_numeric_data().fillna(0) 
df_optcat = train.copy() ; df_optcat = thewholeshabang(df_optcat, df_optcat['SalePrice'], "train")
df_qcat = train.copy() ; df_qcat = nafix(df_qcat)

df_dumcat.to_csv('/content/drive/My Drive/ml_project/train_dumcat.csv')
df_nocat.to_csv('/content/drive/My Drive/ml_project/train_nocat.csv')
df_optcat.to_csv('/content/drive/My Drive/ml_project/train_optcat.csv')
#df_qcat.to_csv('/content/drive/My Drive/ml_project/train_qcat.csv')


####################
#### TEST DFS ######
####################
df = test.copy(); df.set_index('Id')

# Go through same feature engineering as train set
df = beefup(df)
df = nafix(df) 
df = q_to_cat(df, ['MSSubClass', 'OverallCond', 'MoSold'])

# Take the suggest list from previosly ran CatAnalysis for TRAIN set
suggestdf = pd.read_csv("/content/drive/My Drive/ml_project/Katy/FeatureSuggestion.csv")
suggestdf = suggestdf.set_index('Unnamed: 0')['suggest']

# split into different feature sets     
quantifycols = list(suggestdf[suggestdf == "quantify"].index.values)
ovacols = list(suggestdf[suggestdf == "1vA"].index.values)
ignorecols = list(suggestdf[suggestdf == "ignore"].index.values)
dumcols = list(suggestdf[suggestdf == 'dummify'].index.values)
dumcols = dumcols + list(suggestdf[suggestdf == 'binary'].index.values)

# perform the suggested action
for col in df:
    if col in quantifycols:
        df[col] = df[col].fillna(0).replace('None', 0).replace('Po', 1).replace('Fa',2).replace('TA', 3).replace('Gd',4).replace('Ex',5)
    elif col in ovacols:
        df[col] = df[col].eq(df[col].mode()[0]).mul(1)
    elif col in ignorecols:
        df[col] = df.drop(col, axis = 1)

df = pd.get_dummies(df, columns = dumcols, drop_first = True)


df_optcat = df.copy()
df_dumcat = pd.get_dummies(df, drop_first = True)
df_nocat = df._get_numeric_data().fillna(0)
df_mattcat = test.copy() ; custom_pre_proc(df_mattcat)
#df_qcat = nafix(df)


df_dumcat.to_csv('/content/drive/My Drive/ml_project/test_dumcat.csv')
df_nocat.to_csv('/content/drive/My Drive/ml_project/test_nocat.csv')
df_optcat.to_csv('/content/drive/My Drive/ml_project/test_optcat.csv')
#df_qcat.to_csv('/content/drive/My Drive/ml_project/test_qcat.csv')

In [0]:
####################
#### FILEPATHS #####
####################
train_orig = '/content/drive/My Drive/ml_project/matt/train.csv'
test_orig = '/content/drive/My Drive/ml_project/matt/test.csv'
train_mattcat = '/content/drive/My Drive/ml_project/train_mattcat.csv'
test_mattcat = '/content/drive/My Drive/ml_project/test_mattcat.csv'
train_dumcat = '/content/drive/My Drive/ml_project/train_nocat.csv'
test_dumcat = '/content/drive/My Drive/ml_project/test_dumcat.csv'
train_nocat = '/content/drive/My Drive/ml_project/train_dumcat.csv'
test_nocat = '/content/drive/My Drive/ml_project/test_nocat.csv'
train_optcat = '/content/drive/My Drive/ml_project/train_optcat.csv'
test_optcat = '/content/drive/My Drive/ml_project/test_optcat.csv'
#train_qcat = '/content/drive/My Drive/ml_project/train_qcat.csv'
#test_qcat = '/content/drive/My Drive/ml_project/test_qcat.csv'

In [12]:
##loading in the pre-processed data
train_df = h2o.import_file(train_mattcat)
test_df = h2o.import_file(test_mattcat)

print(train_df.shape)
print(test_df.shape)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
(1460, 147)
(1459, 147)


In [13]:
#h2o has it's own eda `describe` function. Uncomment to view
x = train_df.drop(['id', 'saleprice'], axis = 1)
x.describe

C1,1stflrsf,asbestos,attachdgarage,bedroomabvgr,bsmtcond,bsmtexposure,bsmtlivqtr,bsmtlivqtrlq,bsmtqual,bsmttotalbath,bsmtunf,bsmtunfsf,centralair,compshgroof,detachdgarage,enc_2ndfloor,enc_alley,enc_fence,enc_pool,enclosedporch,ext_uniqueness,extercond,exterqual,extrarooms,finishbsmt,finishbsmtsf,finishedstories,finishgarage,finishsf,fireplacequ,garagearea,garagecars,garagecond,garagequal,garageyrblt,gasheat,grlivarea,heatingqc,isartery,isfeeder,kitchenabvgr,kitchenqual,log_price,lotarea,lotfrontage,lotshapereg,lowqualfinsf,lqfinishpct,lqfuse,masvnrarea,miscval,multfireplaces,nobsmt,nogarage,normcond,onext,overallcond,overallqual,paveddrive,plannedunitdev,porchdecksf,positadj,positnear,publicutil,remodgarage,remodhome,residentzone,reszonehidens,rradj,rrnearby,sbreaker,splitmulti,totalbath,totalbsmtsf,totalsf,totrmsabvgrd,typfunc,unique_ext,yardsf,yearbuilt,yearremodadd,bldgtype_2fmCon,bldgtype_Duplex,bldgtype_Twnhs,bldgtype_TwnhsE,foundation_Hip,foundation_Other,landcontour_HLS,landcontour_Low,landcontour_Lvl,landslope_Mod,landslope_Sev,lotconfig_CulDSac,lotconfig_FR2P,lotconfig_Inside,mosold_10,mosold_11,mosold_12,mosold_2,mosold_3,mosold_4,mosold_5,mosold_6,mosold_7,mosold_8,mosold_9,neighborhood_Blueste,neighborhood_BrDale,neighborhood_BrkSide,neighborhood_ClearCr,neighborhood_CollgCr,neighborhood_Crawfor,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_IDOTRR,neighborhood_MeadowV,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NPkVill,neighborhood_NWAmes,neighborhood_NoRidge,neighborhood_NridgHt,neighborhood_OldTown,neighborhood_SWISU,neighborhood_Sawyer,neighborhood_SawyerW,neighborhood_Somerst,neighborhood_StoneBr,neighborhood_Timber,neighborhood_Veenker,roofstyle_Hip,roofstyle_Other,salecondition_Other,salecondition_Partial,saletype_Other,saletype_WD,seassold_Spring,seassold_Summer,seassold_Winter,street_Pave,yrsold_2007,yrsold_2008,yrsold_2009,yrsold_2010
0,856,False,True,3,2,1,True,False,3,1,False,150,True,True,False,True,0,0,0,0,0.343836,2,3,4,True,706,2,False,2416,1,548,2,2,2,2003,True,1710,4,False,False,1,3,12.2477,8450,65,True,0,0,False,1,0,False,False,False,True,True,1,7,True,False,61,False,False,1,False,False,True,False,False,False,False,False,3,856,2566,8,True,False,6679,2003,2003,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0
1,1262,False,True,3,2,4,True,False,3,1,False,284,True,True,False,False,0,0,0,0,0.145205,2,2,2,True,978,1,False,2240,2,460,2,2,2,1976,True,1262,4,False,True,1,2,12.109,9600,80,True,0,0,False,0,0,False,False,False,True,True,4,6,True,False,298,False,False,1,False,False,True,False,False,False,False,False,2,1262,2524,6,True,False,8040,1976,1976,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0
2,920,False,True,3,2,2,True,False,3,1,False,434,True,True,False,True,0,0,0,0,0.343836,2,3,2,True,486,2,False,2272,2,608,2,2,2,2001,True,1786,4,False,False,1,3,12.3172,11250,68,False,0,0,False,1,0,False,False,False,True,True,1,7,True,False,42,False,False,1,False,True,True,False,False,False,False,False,3,920,2706,6,True,False,9422,2001,2002,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0
3,961,False,False,3,3,1,True,False,2,1,False,540,True,True,True,True,0,0,0,272,0.00616438,2,2,3,True,216,2,False,1933,3,642,3,2,2,1998,True,1717,3,False,False,1,3,11.8494,9550,60,False,0,0,False,0,0,False,False,False,True,False,1,7,True,False,35,False,False,1,True,True,True,False,False,False,False,False,1,756,2473,7,True,True,7798,1915,1970,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0
4,1145,False,True,4,2,3,True,False,3,1,False,490,True,True,False,True,0,0,0,0,0.343836,2,3,4,True,655,2,False,2853,2,836,3,2,2,2000,True,2198,4,False,False,1,3,12.4292,14260,84,False,0,0,False,1,0,False,False,False,True,Tru

<bound method H2OFrame.describe of >

In [0]:
test_df.describe

In [14]:
#setting target variable

y = "log_price"


#using h2o's split_frame() to create 80/20 train-validation split
splits = x.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
val = splits[1]
val.describe

C1,1stflrsf,asbestos,attachdgarage,bedroomabvgr,bsmtcond,bsmtexposure,bsmtlivqtr,bsmtlivqtrlq,bsmtqual,bsmttotalbath,bsmtunf,bsmtunfsf,centralair,compshgroof,detachdgarage,enc_2ndfloor,enc_alley,enc_fence,enc_pool,enclosedporch,ext_uniqueness,extercond,exterqual,extrarooms,finishbsmt,finishbsmtsf,finishedstories,finishgarage,finishsf,fireplacequ,garagearea,garagecars,garagecond,garagequal,garageyrblt,gasheat,grlivarea,heatingqc,isartery,isfeeder,kitchenabvgr,kitchenqual,log_price,lotarea,lotfrontage,lotshapereg,lowqualfinsf,lqfinishpct,lqfuse,masvnrarea,miscval,multfireplaces,nobsmt,nogarage,normcond,onext,overallcond,overallqual,paveddrive,plannedunitdev,porchdecksf,positadj,positnear,publicutil,remodgarage,remodhome,residentzone,reszonehidens,rradj,rrnearby,sbreaker,splitmulti,totalbath,totalbsmtsf,totalsf,totrmsabvgrd,typfunc,unique_ext,yardsf,yearbuilt,yearremodadd,bldgtype_2fmCon,bldgtype_Duplex,bldgtype_Twnhs,bldgtype_TwnhsE,foundation_Hip,foundation_Other,landcontour_HLS,landcontour_Low,landcontour_Lvl,landslope_Mod,landslope_Sev,lotconfig_CulDSac,lotconfig_FR2P,lotconfig_Inside,mosold_10,mosold_11,mosold_12,mosold_2,mosold_3,mosold_4,mosold_5,mosold_6,mosold_7,mosold_8,mosold_9,neighborhood_Blueste,neighborhood_BrDale,neighborhood_BrkSide,neighborhood_ClearCr,neighborhood_CollgCr,neighborhood_Crawfor,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_IDOTRR,neighborhood_MeadowV,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NPkVill,neighborhood_NWAmes,neighborhood_NoRidge,neighborhood_NridgHt,neighborhood_OldTown,neighborhood_SWISU,neighborhood_Sawyer,neighborhood_SawyerW,neighborhood_Somerst,neighborhood_StoneBr,neighborhood_Timber,neighborhood_Veenker,roofstyle_Hip,roofstyle_Other,salecondition_Other,salecondition_Partial,saletype_Other,saletype_WD,seassold_Spring,seassold_Summer,seassold_Winter,street_Pave,yrsold_2007,yrsold_2008,yrsold_2009,yrsold_2010
2,920,False,True,3,2,2,True,False,3,1,False,434,True,True,False,True,0,0,0,0,0.343836,2,3,2,True,486,2,False,2272,2,608,2,2,2,2001,True,1786,4,False,False,1,3,12.3172,11250,68,False,0,0,False,1,0,False,False,False,True,True,1,7,True,False,42,False,False,1,False,True,True,False,False,False,False,False,3,920,2706,6,True,False,9422,2001,2002,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0
9,1077,False,True,2,2,1,True,False,2,1,False,140,True,True,False,False,0,0,0,0,0.145205,2,2,1,True,851,1,False,1928,2,205,1,2,3,1939,True,1077,4,True,False,2,2,11.6784,7420,50,True,0,0,False,0,0,True,False,False,False,True,2,5,True,False,4,False,False,1,False,True,True,False,False,False,False,False,1,991,2068,5,True,False,6339,1939,1950,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0
20,1158,False,True,4,2,3,False,False,4,0,False,1158,True,True,False,True,0,0,0,0,0.343836,2,3,4,False,0,2,False,2376,3,853,3,2,2,2005,True,2376,4,False,False,1,3,12.6925,14215,101,False,0,0,False,1,0,False,False,False,True,True,1,8,True,False,394,False,False,1,False,True,True,False,False,False,False,False,4,1158,3534,9,True,False,11445,2005,2006,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
37,1297,False,True,3,2,1,False,False,2,1,False,84,True,True,False,False,0,0,0,0,0.121233,2,2,1,True,1213,1,True,2510,2,498,2,2,2,1954,True,1297,3,False,False,1,2,11.9382,8532,74,True,0,0,False,1,0,False,False,False,True,True,2,5,True,False,0,False,False,1,False,True,True,False,False,False,False,False,1,1297,2594,5,True,False,7235,1954,1990,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0
39,1152,True,False,2,1,0,False,False,0,0,False,0,False,True,False,False,0,0,0,0,0.00136986,2,2,2,False,0,1,False,1152,1,0,0,0,0,1955,True,1152,2,False,False,2,1,11.3145,6040,65,True,0,0,True,0,0,False,True,True,True,Fal

<bound method H2OFrame.describe of >

In [0]:
#####################
##### TRAIN #########
#####################
aml = H2OAutoML(max_runtime_secs = 3600, project_name = "kaggle_iowahousing_automl")
aml.train(y = y, training_frame = train, leaderboard_frame = val)

#h2o does stratified k-fold validation with k = 5 by default

AutoML progress: |██

In [0]:
#####################
##### OBSERVE #######
#####################
aml.leaderboard

In [0]:
#####################
##### FIT ###########
#####################
from scipy.special import boxcox, inv_boxcox1p
pred = aml.predict(val)
pred = pred.as_data_frame().as_matrix()
pred = np.exp(pred)
pred_val = pred.copy()

In [0]:
aml.leader.coef

In [0]:
#####################
##### FIT ###########
#####################
from scipy.special import boxcox, inv_boxcox1p
pred = aml.predict(test_df)
pred = pred.as_data_frame().as_matrix()
pred = np.exp(pred)
pred_test = pred.copy()

submission = pd.DataFrame(columns = ['Id', 'SalePrice'])
saleprice = pred_test


In [0]:
test = pd.read_csv(test_orig)
submission['Id'] = test['Id']
submission['SalePrice'] = saleprice
submission.set_index('Id', inplace = True)
submission.reset_index()
submission.to_csv('/content/drive/My Drive/ml_project/matt/h2o_mattcat.csv')
submission
set(submission.SalePrice)

In [0]:
#####################
##### ANALYZE #######
#####################
perf = aml.leader.model_performance(val)
perf

In [0]:
actual = val[:, ['saleprice']].as_data_frame().as_matrix()
plt.scatter(np.exp(pred_val), np.exp(actual), alpha = .8)
plt.xlabel('Predicted Log Price')
plt.ylabel('Actual Log Price')